# VISUALS
Some visuals we can create using the data. 
## TODO:
* Sanitize inputs (e.g. remove Pro Bowl and All-Pro designation symbols)
    * Sanitize inputs by ignoring non-qualifiers (e.g. receivers who have <20 recs)
* Calculate difference of QB's stats after we remove stats from their top receiver
    * Calculate average difference in yds, TDs, rtg, etc.
* Use data only for QB's who qualify (e.g. >= 300 atts in a season)
* Use data only for receivers who qualify (e.g. top 2/3 leaders in receptions per team)
    * How can we use JOIN to extract this data?
        * How can we then JOIN this data with QB's team?

In [5]:
# IMPORTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
'''
# Get CSVs and store them in a list
advPassAir = []
advPassAccuracy = []
passing = []
rec = []
advRec = []

years = [18, 19, 20, 21, 22, 23]
for x in years:
    advPassAir.append(pd.read_csv('Data/passing/advPassAir' + str(x) + '.csv'))
    advPassAccuracy.append(pd.read_csv('Data/passing/advPassAccuracy' + str(x) + '.csv'))
    passing.append(pd.read_csv('Data/passing/pass' + str(x) + '.csv'))
    rec.append(pd.read_csv('Data/receiving/rec' + str(x) + '.csv'))
    advRec.append(pd.read_csv('Data/receiving/advRec' + str(x) + '.csv'))

print(advPassAir[0].head())
print(advPassAccuracy[0].head())
print(advRec[0].head())
'''

In [49]:

# Get CSVs and store them in a dictionary
advPassAir = {}
advPassAccuracy = {}
passing = {}
rec = {}
advRec = {}

years = [18, 19, 20, 21, 22, 23]
for x in years:
    advPassAir[x] = pd.read_csv('Data/passing/advPassAir' + str(x) + '.csv')
    advPassAccuracy[x] = pd.read_csv('Data/passing/advPassAccuracy' + str(x) + '.csv')
    passing[x] = pd.read_csv('Data/passing/pass' + str(x) + '.csv')
    rec[x] = pd.read_csv('Data/receiving/rec' + str(x) + '.csv')
    advRec[x] = pd.read_csv('Data/receiving/advRec' + str(x) + '.csv')

print(passing)
print(advPassAir[18].head())
print(advPassAccuracy[18].head())
print(advRec[18].head())


{18:      Unnamed: 0   Rk              Player   Tm Age Pos   G  GS   QBrec  Cmp  \
0             0    1  Ben Roethlisberger  PIT  36  QB  16  16   9-6-1  452   
1             1    2   Patrick Mahomes*+  KAN  23  QB  16  16  12-4-0  383   
2             2    3           Matt Ryan  ATL  33  QB  16  16   7-9-0  422   
3             3    4         Jared Goff*  LAR  24  QB  16  16  13-3-0  364   
4             4    5        Andrew Luck*  IND  29  QB  16  16  10-6-0  430   
..          ...  ...                 ...  ...  ..  ..  ..  ..     ...  ...   
104         104  102   DeAndre Hopkins*+  HOU  26  WR  16  16     NaN    0   
105         105  103           Zay Jones  BUF  23  WR  16  15     NaN    0   
106         106  104       Kyle Lauletta  NYG  23  QB   2   0     NaN    0   
107         107  105            JK Scott  GNB  23   P  16   0     NaN    0   
108         108  106      Dede Westbrook  JAX  25  WR  16   9     NaN    0   

     ...    Y/G   Rate   QBR  Sk Yds.1  Sk%  NY/A  ANY/A  

In [93]:
# Sanataize inputs: delete rows with no numeric values in passing data
# Profoobtallreference liks to repeat the row of features within the csv
# several times, however this row is treated like a normal value within
# the dataframe. This loop will see that if 'Att' is a value in the 'Att'  
# feature, then we will drop the row.

for x in years:
    df_drop = passing[x]
    index_to_drop = df_drop[df_drop['Att'] == 'Att'].index
    passing[x].drop(index_to_drop, inplace = True)
    
''' The following doesn't work:
for y in range(len(passing[18]['Att'])):
    print(passing[18]['Att'][y])

However, this works:
for y in passing[18]['Att']:
    print(y)

Why?
'''

# TODO: We need to actually convert the numeric columns from string to int

" The following doesn't work:\nfor y in range(len(passing[18]['Att'])):\n    print(passing[18]['Att'][y])\n\nHowever, this works:\nfor y in passing[18]['Att']:\n    print(y)\n\nWhy?\n"

In [96]:
# Map QB and Receiving Data in 2018

# Get qualifying QBs (e.g. QBs >= 300 pass attempts)
df18 = passing[18]
df18 = df18[df18['Att'] >= 300]


TypeError: '>=' not supported between instances of 'str' and 'int'